In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
import datetime

from functools import reduce

import time 
from IPython.display import clear_output


In [ ]:
#Read CSV file containing the agents from SPENSER
df_persons_SPENSER__dir = r'' # use your path
df_persons_SPENSER__file = os.path.join(df_persons_SPENSER__dir, "")
df_persons_SPENSER = pd.read_csv(df_persons_SPENSER__file, index_col=None, header=0)

In [ ]:
#Read CSV file containing the active employed people aged >=16 per MSOA level 
LC6107EW_MSOA_dir = r'' # use your path
LC6107EW_MSOA_file = os.path.join(LC6107EW_MSOA_dir, "")
df_LC6107EW = pd.read_csv(LC6107EW_MSOA_file, index_col=None, header=0)

In [ ]:
df_LC6107EW.dtypes

In [ ]:
## Keep only relevant columns
df_LC6107EW = df_LC6107EW[['geography_code','people']]

In [ ]:
#Read CSV file 
O_D_msoa_NE_dir = r'' # use your path
O_D_msoa_NE_file = os.path.join(O_D_msoa_NE_dir, "")
df_O_D = pd.read_csv(O_D_msoa_NE_file, index_col=None, header=0)

In [ ]:
df_O_D.head()

In [ ]:
df_O_D.dtypes

In [ ]:
#Read CSV file containing the distances between msoa centroids (meters and miles) 
MSOA_lelvel_centroid_distances_dir = r'' # use your path
MSOA_lelvel_centroid_distances_file = os.path.join(MSOA_lelvel_centroid_distances_dir, "")
df_MSOA_lelvel_centroid_distances = pd.read_csv(MSOA_lelvel_centroid_distances_file, index_col=None, header=0)

In [ ]:
df_MSOA_lelvel_centroid_distances.head()

In [ ]:
# Merge both dataframes to get the distances between MSOA centroids (meter and miles)
# This distances were calculated using the road network
df_O_D = pd.merge(df_O_D, df_MSOA_lelvel_centroid_distances,  how='left', left_on=['Origin','Destination'], right_on = ['msoa_origin', 'msoa_destination'])

In [ ]:
# Remove columns
df_O_D.drop(['msoa_origin', 'msoa_destination', 'distance_meters'], axis=1, inplace=True)

In [ ]:
df_O_D.head()

In [ ]:
#Read CSV file containing the matches between NTS and SPENSER:
NTS_SPENSER_matched_dir = r'' # use your path
NTS_SPENSER_matched_file = os.path.join(NTS_SPENSER_matched_dir, "")
df_NTS_SPENSER_matched = pd.read_csv(NTS_SPENSER_matched_file, index_col=None, header=0)

In [ ]:
df_NTS_SPENSER_matched.dtypes

In [ ]:
len(df_NTS_SPENSER_matched)

In [ ]:
# check the number of null values in the columns:
df_NTS_SPENSER_matched.isnull().sum(axis = 0)

In [ ]:
#Read CSV file containing the selected NTS days:
NTS_days_dir = r'' # use your path
NTS_days_file = os.path.join(NTS_days_dir, "")
df_NTS_days = pd.read_csv(NTS_days_file, index_col=None, header=0)

In [ ]:
len(df_NTS_days)

In [ ]:
# Create a list with all Days unique ID values
days_selected_list = df_NTS_days['DayID'].unique().tolist()

In [ ]:
# Import the file with the days related to each individual
trips_dir = r'' # use your path


df_NTS_trips = pd.read_csv(
    trips_dir,
    sep='\t',
    usecols=['TripID',
             'DayID',               # ID given to all trips made by an individual on a given travel day - Created in SQL
             'IndividualID',        # Individual unique ID - Created in SQL
             'HouseholdID',         # Household unique ID - Created in SQL
             'PSUID',               # PSU unique ID - Created in SQL
             'JourSeq',             # Journey number on a given travel day
             'NumStages',           # Number of stages - actual number
             'MainMode_B04ID',      # Main mode of travel - publication table breakdown - 13 categories
             'TripPurpTo_B01ID',   # Trip purpose - full list - 23 categories
             'TripStartHours',      # Trip start time - hours component
             'TripStartMinutes',    # Trip start time - minutes component
             'TripStart',           # Trip start time - minutes past midnight
             'TripEndHours',        # Trip end time - hours component
             'TripEndMinutes',      # Trip end time - minutes component
             'TripEnd',             # Trip end time - minutes past midnight
             'TripDisIncSW',        # Trip distance - including short walk - miles - actual distance
             'TripTotalTime'        # Total trip time - minutes - actual time
             ]          
)
#persons_in.head()

In [ ]:
len(df_NTS_trips)

In [ ]:
# Keep only those days that belong to the people selected before
df_NTS_trips = df_NTS_trips.loc[(df_NTS_trips['DayID'].isin(days_selected_list))]


In [ ]:
len(df_NTS_trips)

In [ ]:
## Update transpor mode
mode_mapping = {
    1: 'walk',
     2: 'bike',
     3: 'car',  #'Car/van driver'
     4: 'car_passenger',  #'Car/van passenger'
     5: 'motorcycle',  
     6: 'car',  #'Other private transport',
     7: 'bus', #Bus in London',
     8: 'bus', #'Other local bus',
     9: 'bus', #'Non-local bus',
     10: 'metro', #'London Underground',
     11: 'train', #'Surface Rail',
     12: 'car',  #'Taxi/minicab',
     13: 'metro', #'Other public transport',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['MainMode_B04ID'] = df_NTS_trips['MainMode_B04ID'].map(mode_mapping)

In [ ]:
transport_modes_list = df_NTS_trips['MainMode_B04ID'].unique().tolist()

In [ ]:
transport_modes_list

In [ ]:
purp_mapping = {
    1: 'work',
     2: 'work',  #'In course of work',
     3: 'education',
     4: 'food_shop',  #'Food shopping',
     5: 'shop',  #'Non food shopping',
     6: 'medical', #'Personal business medical',
     7: 'eat',  #'Personal business eat/drink',
     8: 'other',  #'Personal business other',
     9: 'eat',  #'Eat/drink with friends',
     10: 'other',  #'Visit friends',
     11: 'leisure',  #'Other social',
     12: 'leisure',  #'Entertain/ public activity',
     13: 'leisure_sport',  #'Sport: participate',
     14: 'home',  #'Holiday: base',
     15: 'leisure_sport',  #'Day trip/just walk',
     16: 'leisure',  #'Other non-escort',
     17: 'escort',  #'Escort home',
     18: 'escort',  #'Escort work',
     19: 'escort',  #'Escort in course of work',
     20: 'escort',  #'Escort education',
     21: 'escort',  #'Escort shopping/personal business',
     22: 'escort',  #'Other escort',
     23: 'home',  #'Home',
     -10: 'DEAD',
     -8: 'NA'
}

In [ ]:
df_NTS_trips['TripPurpTo_B01ID'] = df_NTS_trips['TripPurpTo_B01ID'].map(purp_mapping)

In [ ]:
df_NTS_trips['TripPurpTo_B01ID'].unique()

In [ ]:
# Select only those individuals that are employed and older than 15 years.
df_persons_SPENSER_employed = df_persons_SPENSER.loc[(df_persons_SPENSER['Age'] >= 16) & (df_persons_SPENSER['Economic_activity'] == 'Employed')]


In [ ]:
len(df_persons_SPENSER_employed)

In [ ]:
file_saved = 'workplace_destination.csv' # in case
O_D_file_saved = 'O_D_msoa_NE_all_updated.csv' # in case
workplace_destination_round2 = 'workplace_destination_round2.csv' # in case
#workplace_destination_round3 = 'workplace_destination_round3.csv' # in case


In [ ]:
individuals_matched_list = []
msoa_list = df_persons_SPENSER['Area_MSOA'].unique()
spenser_id_list = []
counter = 0

for msoa_code in msoa_list:
    
    
    counter += 1
    clear_output(wait=True)
    print(f' {counter} of {len(msoa_list)}, MSOA name: {msoa_code}')
    print(msoa_code)
    
    ## Identify those agents in the MSOA level, (previously selected: older than 15 and employed)
    df_persons_SPENSER_msoa = df_persons_SPENSER_employed.loc[(df_persons_SPENSER_employed['Area_MSOA']== msoa_code) & ~(df_persons_SPENSER_employed['PID_AreaMSOA'].isin(spenser_id_list)) ]
    
    # List the IDs of the agents selected
    persons_ID_SPENSER_msoa_list = df_persons_SPENSER_msoa['PID_AreaMSOA'].unique().tolist()
    
    # count the number of agents selected
    agents_2019_selected = len(df_persons_SPENSER_msoa)
    

    ## Identify those people (census 2011) in the MSOA level that are employed and older than 16 years old
    df_LC6107EW_msoa = df_LC6107EW.loc[(df_LC6107EW['geography_code'] == msoa_code)]
    
    # Get the number of people that are employed and older than 15 years old in the selected MSOA level
    people_2011_selected = df_LC6107EW_msoa['people'].values[0]
    

    # Calculate the ratio of people between 2019 and 2011 per MSOA level
    ## This value is to update the number of people travelling in the NE from the selected MSOA level
    ### We assume a linear incrase or decrase of people using each transport mode from 2011 to 2019
    ratio_people_2019_2011 = agents_2019_selected / people_2011_selected
    
    

    #Select the rows of the O_D_matrix that is related to the selected msoa level:
    df_O_D_msoa = df_O_D.loc[(df_O_D['Origin'] == msoa_code)]
    

    ## update the values to be assinged from 2011 to 2019!!
    df_O_D_msoa['car'] = round((df_O_D_msoa['car'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['bus'] = round((df_O_D_msoa['bus'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['bike'] = round((df_O_D_msoa['bike'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['walk'] = round((df_O_D_msoa['walk'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['motorcycle'] = round((df_O_D_msoa['motorcycle'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['car_passenger'] = round((df_O_D_msoa['car_passenger'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['train'] = round((df_O_D_msoa['train'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['metro'] = round((df_O_D_msoa['metro'] * ratio_people_2019_2011),0).astype(int)
    df_O_D_msoa['total'] = df_O_D_msoa['car'] + df_O_D_msoa['bus'] + df_O_D_msoa['bike'] + df_O_D_msoa['walk'] + df_O_D_msoa['motorcycle'] + df_O_D_msoa['car_passenger'] + df_O_D_msoa['train'] + df_O_D_msoa['metro']
    
    
    # save the updated data in a csv file
    df_O_D_msoa.to_csv(O_D_file_saved, encoding='utf-8', mode='a', index=False, header=False)
            

    # Sort df_O_D_msoa by distance (shorter distances in top)
    df_O_D_msoa = df_O_D_msoa.sort_values(['distance_miles'], ascending = [True])
    
    ################################
    

    
    ## loop through each of the transport modes and assign each individual a destination:
    for transport_mode in transport_modes_list:

        ## Select those matched individuals between NTS ns SPENSER and 
        df_NTS_SPENSER_matched_msoa = df_NTS_SPENSER_matched.loc[(df_NTS_SPENSER_matched['PID_AreaMSOA'].isin(persons_ID_SPENSER_msoa_list)) & ~ (df_NTS_SPENSER_matched['PID_AreaMSOA'].isin(spenser_id_list))]
        
        # List the IDs of the agents selected
        persons_ID_NTS_msoa_list = df_NTS_SPENSER_matched_msoa['IndividualID'].unique().tolist()
    
        # list of DayID of the agents selected
        days_ID_NTS_msoa_list = df_NTS_SPENSER_matched_msoa['DayID'].unique().tolist()
    
        
        
        ## Select only those trips that belong to the people and days selected before +  purpose = work + mode = transport_mode
        df_NTS_trips_selected = df_NTS_trips.loc[((df_NTS_trips['IndividualID'].isin(persons_ID_NTS_msoa_list)) & (df_NTS_trips['TripPurpTo_B01ID'] == 'work') & (df_NTS_trips['DayID'].isin(days_ID_NTS_msoa_list)) & (df_NTS_trips['MainMode_B04ID'] == transport_mode))]  
        
        
        ## Remove duplicate rows based on IndividualID and DayID (if they have more than one trip for purpose =work, keep only the first)
        df_NTS_trips_selected = df_NTS_trips_selected.drop_duplicates(subset=['IndividualID', 'DayID'], keep="first")
        
        ## Create a new dataframe containing only the relevant columns
        df_NTS_trips_selected_short = df_NTS_trips_selected[['IndividualID', 'DayID', 'TripDisIncSW', 'MainMode_B04ID', 'TripPurpTo_B01ID']]
        

        ## Merge dataframes to combine df_NTS_SPENSER_matched_msoa with the distances travelled by each individual        
        df_NTS_SPENSER_matched_msoa_transportmode = pd.merge(df_NTS_SPENSER_matched_msoa, df_NTS_trips_selected_short,  how='left', left_on=['IndividualID','DayID'], right_on = ['IndividualID','DayID'])
        
        ## Drop those rows where there are null values
        df_NTS_SPENSER_matched_msoa_transportmode = df_NTS_SPENSER_matched_msoa_transportmode.dropna(axis=0, subset=['MainMode_B04ID'])
        
        ## Keep only relevant columns
        df_NTS_SPENSER_matched_msoa_transportmode = df_NTS_SPENSER_matched_msoa_transportmode[['PID_AreaMSOA', 'IndividualID', 'DayID', 'TripDisIncSW', 'MainMode_B04ID', 'TripPurpTo_B01ID']]
        
        ## Sort values by distance
        df_NTS_SPENSER_matched_msoa_transportmode = df_NTS_SPENSER_matched_msoa_transportmode.sort_values(['TripDisIncSW'], ascending = [True])
        
        # At this stage we have identified the spenser people (msoa level, aged >=16 and employed) that travel to work by "transport_mode" selected in the previous 'for loop'
        
        
        ########
        ## Now we need to match them msoa level they go based on the transport mode they use
        ### loop through the df_O_D_msoa matrix and select the value of the column related to the transport mode selected in the first for loop
        

        
        counter_people_selected_for_transport_mode = 0
        
        for idx_msoa, msoa_row in df_O_D_msoa.iterrows():
            
            # Select the people that travel by the transpor_mode selected in the previous for loop
            people_in_transport_mode = msoa_row[transport_mode]
            
            
            msoa_destination = msoa_row['Destination']
            

            # Select the top "x" values
            # If there are more people in the synthetic population than in the O_C_matrix:
            if (len(df_NTS_SPENSER_matched_msoa_transportmode) >= people_in_transport_mode):
            
            
                df_NTS_SPENSER_matched_msoa_transportmode_matched = df_NTS_SPENSER_matched_msoa_transportmode.head(people_in_transport_mode)
            
            # If there are less, select the remaining ones           
            else:
                
                df_NTS_SPENSER_matched_msoa_transportmode_matched = df_NTS_SPENSER_matched_msoa_transportmode
            

            counter_people_selected_for_transport_mode += len(df_NTS_SPENSER_matched_msoa_transportmode_matched)
            
            # Concatenate the selected ones with the people from the same age and OA area
            df_concatenated = (pd.concat([df_NTS_SPENSER_matched_msoa_transportmode,df_NTS_SPENSER_matched_msoa_transportmode_matched]))
            
            
            #Remove duplicates BUT keep the same names of the dataframes used after selecting the OAarea, age and sex
            df_NTS_SPENSER_matched_msoa_transportmode = df_concatenated.drop_duplicates(subset='PID_AreaMSOA', keep = False)
            
            ## update the value
            df_NTS_SPENSER_matched_msoa_transportmode_matched['Origin'] = msoa_code
            
            ## update the value
            df_NTS_SPENSER_matched_msoa_transportmode_matched['Destination'] = msoa_destination
            
            # update the value
            df_NTS_SPENSER_matched_msoa_transportmode_matched['O_D_transport_mode'] = transport_mode
            
            #Remove duplicates BUT keep the same names of the dataframes used after selecting the OAarea, age and sex
            df_NTS_SPENSER_matched_msoa_transportmode_matched = df_NTS_SPENSER_matched_msoa_transportmode_matched.drop_duplicates(subset='PID_AreaMSOA', keep = 'first')
            

            
            ## loop through the people selected and save them in a list
            for idx_person_matched, person_matched in df_NTS_SPENSER_matched_msoa_transportmode_matched.iterrows():
            
                spenser_id_list.append(person_matched['PID_AreaMSOA'])
            
            
            
            
            
            
            
            # save the data in a csv file
            df_NTS_SPENSER_matched_msoa_transportmode_matched.to_csv(file_saved, encoding='utf-8', mode='a', index=False, header=False)
            


print('Code has been finished. Check results!')

In [ ]:
header =["PID_AreaMSOA", "IndividualID", "DayID", "TripDisIncSW", "MainMode_B04ID", "TripPurpTo_B01ID", "Origin", "Destination", "O_D_transport_mode"]

In [ ]:
#Read CSV file containing the agents from SPENSER
df_NTS_SPENSER_matched_msoa_transportmode_matched_dir = r'' # use your path
df_NTS_SPENSER_matched_msoa_transportmode_matched_file = os.path.join(df_NTS_SPENSER_matched_msoa_transportmode_matched_dir, "workplace_destination.csv")
df_NTS_SPENSER_matched_msoa_transportmode_matched = pd.read_csv(df_NTS_SPENSER_matched_msoa_transportmode_matched_file, index_col=None, header=None, names= header)

In [ ]:
len(df_NTS_SPENSER_matched_msoa_transportmode_matched)

In [ ]:
df_NTS_SPENSER_matched_msoa_transportmode_matched

In [ ]:
df_grouped_first_results = pd.DataFrame({'count' : df_NTS_SPENSER_matched_msoa_transportmode_matched.groupby( ['Origin','Destination'] ).size()}).reset_index()

In [ ]:
df_grouped_first_results

In [ ]:
df_grouped_first_results['count'].sum()

In [ ]:
header_2 = ["Origin", "Destination", "car", "bus", "bike", "walk", "motorcycle", "car_passenger", "train", "metro", "total", "distance_miles"]

In [ ]:
#Read CSV file containing the agents from SPENSER
O_D_msoa_NE_updated_dir = r'' # use your path
O_D_msoa_NE_updated_file = os.path.join(O_D_msoa_NE_updated_dir, "O_D_msoa_NE_all_updated.csv")
df_O_D_msoa_NE_updated = pd.read_csv(O_D_msoa_NE_updated_file, index_col=None, header=None, names= header_2)

In [ ]:
df_O_D_msoa_NE_updated

In [ ]:
df_O_D_msoa_NE_updated['total'].sum()

In [ ]:
# Merge both dataframes
df_O_D_msoa_NE_updated_merged = pd.merge(df_O_D_msoa_NE_updated, df_grouped_first_results,  how='left', left_on=['Origin', 'Destination'], right_on = ['Origin', 'Destination'])

In [ ]:
df_O_D_msoa_NE_updated_merged

In [ ]:
# Repalce Nan values by 0
df_O_D_msoa_NE_updated_merged = df_O_D_msoa_NE_updated_merged.fillna(0)

In [ ]:
# Generate a new column with the remaining number of people to be assigned to each MSOA, based on data from the O_D matrix 2011
df_O_D_msoa_NE_updated_merged['Remaining'] = df_O_D_msoa_NE_updated_merged['total'] - df_O_D_msoa_NE_updated_merged['count']

In [ ]:
df_O_D_msoa_NE_updated_merged

In [ ]:
# Keep only relevant columns
df_O_D_msoa_NE_updated_merged = df_O_D_msoa_NE_updated_merged[['Origin', 'Destination', 'total','Remaining', 'distance_miles']]

In [ ]:
df_O_D_msoa_NE_updated_merged

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].sum()

In [ ]:
## Keep only those columns when Remaining > 0
df_O_D_msoa_NE_updated_merged = df_O_D_msoa_NE_updated_merged.loc[df_O_D_msoa_NE_updated_merged['Remaining'] > 0]

In [ ]:
df_O_D_msoa_NE_updated_merged.head()

In [ ]:
len(df_O_D_msoa_NE_updated_merged)

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].sum()

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].min()

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].max()

In [ ]:
## Identify those spenser people that have not been selected a MSOA workplace yet.
spenser_id_already_selected_list = df_NTS_SPENSER_matched_msoa_transportmode_matched['PID_AreaMSOA'].unique().tolist()


df_persons_SPENSER_employed_remaining = df_persons_SPENSER_employed.loc[(~df_persons_SPENSER_employed['PID_AreaMSOA'].isin(spenser_id_already_selected_list))]


In [ ]:
len(df_persons_SPENSER_employed_remaining)

In [ ]:
counter = 0

for msoa_code in msoa_list:
    
    # If there is still at least one row available to be assigned to an employed, do the following.
    ## else: break
    if (len(df_O_D_msoa_NE_updated_merged) > 0):
    
        counter += 1
        clear_output(wait=True)
        print(f' {counter} of {len(msoa_list)}, MSOA name: {msoa_code}')

        ## Identify those remaining agents in the MSOA level
        df_persons_SPENSER_msoa = df_persons_SPENSER_employed_remaining.loc[(df_persons_SPENSER_employed_remaining['Area_MSOA']== msoa_code)]


        # List the IDs of the agents selected
        persons_ID_SPENSER_msoa_list = df_persons_SPENSER_msoa['PID_AreaMSOA'].unique().tolist()



        ## Select those matched individuals between NTS and SPENSER  
        df_NTS_SPENSER_matched_msoa = df_NTS_SPENSER_matched.loc[(df_NTS_SPENSER_matched['PID_AreaMSOA'].isin(persons_ID_SPENSER_msoa_list))]

        # List the IDs of the agents selected
        persons_ID_NTS_msoa_list = df_NTS_SPENSER_matched_msoa['IndividualID'].unique().tolist()

        # list of DayID of the agents selected
        days_ID_NTS_msoa_list = df_NTS_SPENSER_matched_msoa['DayID'].unique().tolist()



        ## Select only those trips that belong to the people and days selected before and trip purpose = 'work'
        df_NTS_trips_selected = df_NTS_trips.loc[((df_NTS_trips['IndividualID'].isin(persons_ID_NTS_msoa_list)) & (df_NTS_trips['DayID'].isin(days_ID_NTS_msoa_list)) & (df_NTS_trips['TripPurpTo_B01ID'] == 'work'))]  

        # Sort df_NTS_trips_selected by distance
        df_NTS_trips_selected = df_NTS_trips_selected.sort_values(['TripDisIncSW'], ascending = [False])

        ## Remove duplicate rows based on IndividualID and DayID (if they have more than one trip, keep only the first (the longest))
        df_NTS_trips_selected = df_NTS_trips_selected.drop_duplicates(subset=['IndividualID', 'DayID'], keep="first")

        ## Create a new dataframe containing only the relevant columns
        df_NTS_trips_selected_short = df_NTS_trips_selected[['IndividualID','DayID', 'TripDisIncSW', 'MainMode_B04ID', 'TripPurpTo_B01ID']]


        ## Merge dataframes to combine df_NTS_SPENSER_matched_msoa with the distances travelled by each individual        
        df_NTS_SPENSER_matched_msoa_remaining = pd.merge(df_NTS_SPENSER_matched_msoa, df_NTS_trips_selected_short,  how='left', left_on=['IndividualID','DayID'], right_on = ['IndividualID','DayID'])



        ## Drop those rows where there are null values (for example, with the column MainMode_B04ID. It means that that individual does not have any trip assigned)
        df_NTS_SPENSER_matched_msoa_remaining = df_NTS_SPENSER_matched_msoa_remaining.dropna(axis=0, subset=['MainMode_B04ID'])

        ## Keep only relevant columns
        df_NTS_SPENSER_matched_msoa_remaining = df_NTS_SPENSER_matched_msoa_remaining[['PID_AreaMSOA', 'IndividualID', 'DayID', 'TripDisIncSW', 'MainMode_B04ID', 'TripPurpTo_B01ID']]

        ## Sort values by distance
        df_NTS_SPENSER_matched_msoa_remaining = df_NTS_SPENSER_matched_msoa_remaining.sort_values(['TripDisIncSW'], ascending = [True])


        ### At this stage, we have the people in the msoa level (origin) that need to be assinged a msoa area for work (destination). based on their distance travelled.
        ############################################


        ## Loop through each individual and match their travelled distance to the closest MSOA area:
        for idx_individual, NTS_SPENSER_individual in df_NTS_SPENSER_matched_msoa_remaining.iterrows():

            ##Get their distance travelled
            individual_distance_travelled = NTS_SPENSER_individual['TripDisIncSW']


            # Get the possible msoa destinations based on the msoa origin value
            df_O_D_msoa_NE_updated_merged_msoa_unique = df_O_D_msoa_NE_updated_merged.loc[(df_O_D_msoa_NE_updated_merged['Origin'] == msoa_code) & (df_O_D_msoa_NE_updated_merged['Remaining'] > 0)]



            if (len(df_O_D_msoa_NE_updated_merged_msoa_unique) > 0):


                # Find the closest MSOA level destination from "df_O_D_msoa_NE_updated_merged", based on the "distance_miles" column
                O_D_msoa_destination_row_index = df_O_D_msoa_NE_updated_merged_msoa_unique['distance_miles'].sub(individual_distance_travelled).abs().idxmin()


                df_O_D_msoa_destination_chosen = df_O_D_msoa_NE_updated_merged_msoa_unique.loc[O_D_msoa_destination_row_index]

                df_O_D_msoa_destination_chosen = df_O_D_msoa_destination_chosen.to_frame().T


                # Identify the current number of people to be assigned this MSOA level to travel based on the O_D matrix
                current_remainig = df_O_D_msoa_destination_chosen.iloc[0]['Remaining']
                
                
                #print(current_remainig)
                

                # Remove one unit of the current remainig people to be assigned to that MSOA level
                ## the person selected in the for loop will take this place and should be not accesible for the others.
                df_O_D_msoa_NE_updated_merged.at[O_D_msoa_destination_row_index,'Remaining'] = current_remainig - 1
                
                
                #print(df_O_D_msoa_NE_updated_merged.at[O_D_msoa_destination_row_index,'Remaining'])
                
                #sys.exit()

                ## Create two new columns in the dataframe
                NTS_SPENSER_individual["Origin"] = df_O_D_msoa_destination_chosen.iloc[0]['Origin']
                NTS_SPENSER_individual["Destination"] = df_O_D_msoa_destination_chosen.iloc[0]['Destination']





                df_NTS_SPENSER_individual = pd.DataFrame([NTS_SPENSER_individual])




                ## Save the individual in a csv file
                df_NTS_SPENSER_individual.to_csv(workplace_destination_round2, encoding='utf-8', mode='a', index=False, header=False)



                ## if the number of remaining people to be assinged to that MSOA level is 0, then drop the column
                if (df_O_D_msoa_NE_updated_merged.at[O_D_msoa_destination_row_index,'Remaining'] == 0):

                    df_O_D_msoa_NE_updated_merged = df_O_D_msoa_NE_updated_merged.drop(labels= O_D_msoa_destination_row_index, axis=0)


    else:
        
        print('All available slots have been assigned. Process is finished.')
        break
            
print('Code has been finished. Check results, amigo!')

In [ ]:
header_3 =["PID_AreaMSOA", "IndividualID", "DayID", "TripDisIncSW", "MainMode_B04ID", "TripPurpTo_B01ID", "Origin", "Destination"]

In [ ]:
#Read CSV file 
workplace_destination_round2_dir = r'' # use your path
workplace_destination_round2_file = os.path.join(workplace_destination_round2_dir, "workplace_destination_round2.csv")
df_workplace_destination_round2 = pd.read_csv(workplace_destination_round2_file, index_col=None, header=None, names= header_3)

In [ ]:
len(df_workplace_destination_round2)

In [ ]:
df_workplace_destination_round2

In [ ]:
#(len(df_NTS_SPENSER_matched_msoa_transportmode_matched) + len(df_workplace_destination_round2)) /len(df_persons_SPENSER_employed) * 100

In [ ]:
#len(df_NTS_SPENSER_matched_msoa_transportmode_matched) + len(df_workplace_destination_round2)

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].sum()

In [ ]:
df_O_D_msoa_NE_updated_merged

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].min()

In [ ]:
df_O_D_msoa_NE_updated_merged['Remaining'].max()

In [ ]:
df_O_D_msoa_NE_updated_merged.loc[df_O_D_msoa_NE_updated_merged['Remaining'] > 100]

In [ ]:
aa = df_O_D_msoa_NE_updated_merged.loc[df_O_D_msoa_NE_updated_merged['Destination'] =='E02001731']

In [ ]:
aa['Remaining'].sum()

In [ ]:
frames = [df_NTS_SPENSER_matched_msoa_transportmode_matched, df_workplace_destination_round2]

result_2_rounds = pd.concat(frames)

In [ ]:
len(result_2_rounds)

In [ ]:
result_2_rounds_short = result_2_rounds

In [ ]:
result_2_rounds_short = result_2_rounds_short[['PID_AreaMSOA', 'IndividualID', 'DayID', 'Origin', 'Destination']]

In [ ]:
result_2_rounds_short

In [ ]:
result_2_rounds_short.to_csv(r'', encoding='utf-8', mode='a', index=False)


In [ ]:
###########################################################################
#####################


###############################################################

In [ ]:
# 1. Calculate the number of employed people per MSOA level

In [ ]:
df_msoa_employed = df_persons_SPENSER_employed.groupby(['Area_MSOA'], sort=False).size().reset_index(name='Count')

In [ ]:
len(df_msoa_employed)

In [ ]:
df_msoa_employed

In [ ]:
# 2. calculate the number of employed people commuting within the NE per MSOA level

In [ ]:
result_2_rounds_short['msoa'] = result_2_rounds_short['PID_AreaMSOA'].str.split('_').str[-1]

In [ ]:
df_msoa_employed_commutingNE = result_2_rounds_short.groupby(['msoa'], sort=False).size().reset_index(name='Count')

In [ ]:
df_msoa_employed_commutingNE

In [ ]:
## Merge dataframes to combine df_NTS_SPENSER_matched_msoa with the distances travelled by each individual        
df_NTS_SPENSER_matched_msoa_remaining = pd.merge(df_NTS_SPENSER_matched_msoa, df_NTS_trips_selected_short,  how='left', left_on=['IndividualID','DayID'], right_on = ['IndividualID','DayID'])


In [ ]:
df_msoa_employed_commutingNE_comp_2019 = pd.merge(df_msoa_employed, df_msoa_employed_commutingNE, how='left', left_on=['Area_MSOA'], right_on = ['msoa'])

In [ ]:
df_msoa_employed_commutingNE_comp_2019

In [ ]:
df_msoa_employed_commutingNE_comp_2019.rename(columns={'Count_x': 'employed_active', 'Count_y': 'people_commuting_NE'}, inplace=True)

In [ ]:
df_msoa_employed_commutingNE_comp_2019

In [ ]:
df_msoa_employed_commutingNE_comp_2019 = df_msoa_employed_commutingNE_comp_2019[['Area_MSOA','employed_active','people_commuting_NE']]

In [ ]:
df_msoa_employed_commutingNE_comp_2019

In [ ]:
df_msoa_employed_commutingNE_comp_2019['percentage_people_commuting_NE_2019'] = round(df_msoa_employed_commutingNE_comp['people_commuting_NE'] / df_msoa_employed_commutingNE_comp['employed_active']  *100,2)

In [ ]:
df_msoa_employed_commutingNE_comp_2019

In [ ]:
df_msoa_employed_commutingNE_comp_2019['people_commuting_NE'].sum()/df_msoa_employed_commutingNE_comp_2019['employed_active'].sum() * 100

In [ ]:
df_msoa_employed_commutingNE_comp_2019['percentage_people_commuting_NE_2019'].min()

In [ ]:
df_msoa_employed_commutingNE_comp_2019['percentage_people_commuting_NE_2019'].max()

In [ ]:
df_msoa_employed_commutingNE_comp_2019['percentage_people_commuting_NE_2019'].median()

In [ ]:
#Read CSV file containing the agents from SPENSER
perc_employed_commuting_NE_2011__dir = r'C:\Users\b9055315\PhD_project\UK_Data_Service\NTS\Generated_data_from_code\workplace_destination' # use your path
perc_employed_commuting_NE_2011__file = os.path.join(perc_employed_commuting_NE_2011__dir, "perc_employed_commuting_NE_2011.csv")
df_perc_employed_commuting_NE_2011 = pd.read_csv(perc_employed_commuting_NE_2011__file, index_col=None, header=0)

In [ ]:
df_perc_employed_commuting_NE_2011

In [ ]:
df_msoa_employed_commutingNE_year_comparisson = pd.merge(df_perc_employed_commuting_NE_2011, df_msoa_employed_commutingNE_comp_2019, how='left', left_on=['msoa_code'], right_on = ['Area_MSOA'])

In [ ]:
df_msoa_employed_commutingNE_year_comparisson = df_msoa_employed_commutingNE_year_comparisson[['msoa_code', 'percentage_people_commuting_NE', 'percentage_people_commuting_NE_2019']]

In [ ]:
df_msoa_employed_commutingNE_year_comparisson

In [ ]:
df_msoa_employed_commutingNE_year_comparisson.rename(columns={'percentage_people_commuting_NE': 'percentage_people_commuting_NE_2011'}, inplace=True)

In [ ]:
df_msoa_employed_commutingNE_year_comparisson

In [ ]:
df_msoa_employed_commutingNE_year_comparisson['percentage_diff'] = df_msoa_employed_commutingNE_year_comparisson['percentage_people_commuting_NE_2011'] - df_msoa_employed_commutingNE_year_comparisson['percentage_people_commuting_NE_2019']

In [ ]:
df_msoa_employed_commutingNE_year_comparisson['percentage_diff'].min()

In [ ]:
df_msoa_employed_commutingNE_year_comparisson['percentage_diff'].max()

In [ ]:
df_msoa_employed_commutingNE_year_comparisson

In [ ]:
df_msoa_employed_commutingNE_year_comparisson.to_csv(r'C:\Users\b9055315\PhD_project\UK_Data_Service\NTS\Generated_data_from_code\workplace_destination\sixth_attempt\msoa_employed_commutingNE_year_comparisson.csv', encoding='utf-8', mode='a', index=False, header=True)


In [ ]:
df_msoa_employed_commutingNE_year_comparisson.sort_values('percentage_diff', ascending=True).head(50)

In [ ]:
df_msoa_employed_commutingNE_year_comparisson.loc[(df_msoa_employed_commutingNE_year_comparisson['percentage_diff'] < 0)]

In [ ]:
a = df_msoa_employed_commutingNE_year_comparisson.loc[(df_msoa_employed_commutingNE_year_comparisson['percentage_diff'] > 0)]

In [ ]:
a['msoa_code'].unique().tolist()